# Streamlit

**OBJECTIVES**

- Save objects with the `pickle` module
- Save `sklearn` models as `.pkl` files
- Build and deploy basic streamlit applications

In [1]:
import warnings
warnings.filterwarnings('ignore')

## Serialization and `pickle`

One approach to writing python objects out is to serialize, or create a byte stream. This is done using the `pickle` module, though other options exist. **Note**: Pickle files are not secure and you should not trust unknown sources of pickled files.

In [2]:
import pickle

In [3]:
v1 = [1, 2, 3]

In [4]:
with open('simple_list.pkl', 'wb') as f:
    pickle.dump(v1, f)

In [5]:
with open('simple_list.pkl', 'rb') as f:
    v2 = pickle.load(f)

In [6]:
v2

[1, 2, 3]

### Example: Regression Model

Below we build and save a pipeline to share with our streamlit app.  `Pipeline` objects will make input and transformations easy and are able to be pickled.  There are security issues with `pickle` and some alternative ideas if looking to use unknown sources for models such as [skops](https://skops.readthedocs.io/en/stable/index.html).

In [7]:
from sklearn.datasets import fetch_openml
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

In [8]:
houses = fetch_openml(data_id = 43926)

In [9]:
data = houses.frame

In [10]:
data.head()

,MS_SubClass,MS_Zoning,Lot_Frontage,Lot_Area,Street,Alley,Lot_Shape,Land_Contour,Utilities,Lot_Config,...,Fence,Misc_Feature,Misc_Val,Mo_Sold,Year_Sold,Sale_Type,Sale_Condition,Sale_Price,Longitude,Latitude
0,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,141,31770,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,None,0,5,2010,WD,Normal,215000,-93.619754,42.054035
1,One_Story_1946_and_Newer_All_Styles,Residential_High_Density,80,11622,Pave,No_Alley_Access,Regular,Lvl,AllPub,Inside,...,Minimum_Privacy,None,0,6,2010,WD,Normal,105000,-93.619756,42.053014
2,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,81,14267,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Corner,...,No_Fence,Gar2,12500,6,2010,WD,Normal,172000,-93.619387,42.052659
3,One_Story_1946_and_Newer_All_Styles,Residential_Low_Density,93,11160,Pave,No_Alley_Access,Regular,Lvl,AllPub,Corner,...,No_Fence,None,0,4,2010,WD,Normal,244000,-93.617320,42.051245
4,Two_Story_1946_and_Newer,Residential_Low_Density,74,13830,Pave,No_Alley_Access,Slightly_Irregular,Lvl,AllPub,Inside,...,Minimum_Privacy,None,0,3,2010,WD,Normal,189900,-93.638933,42.060899


In [11]:
X = data[['Gr_Liv_Area', 'Overall_Qual', 'Sale_Condition', 'Lot_Area']]
y = data['Sale_Price']

In [12]:
transformer = make_column_transformer((OneHotEncoder(), X.select_dtypes('category').columns.tolist()),
                                      remainder = 'passthrough')

In [13]:
model = LinearRegression()

In [14]:
pipeline = Pipeline([('transformer', transformer), ('model', model)])

In [15]:
pipeline.fit(X, y)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(),
                                                  ['Overall_Qual',
                                                   'Sale_Condition'])])),
                ('model', LinearRegression())])

In [16]:
X.head()

,Gr_Liv_Area,Overall_Qual,Sale_Condition,Lot_Area
0,1656,Above_Average,Normal,31770
1,896,Average,Normal,11622
2,1329,Above_Average,Normal,14267
3,2110,Good,Normal,11160
4,1629,Average,Normal,13830


In [17]:
with open('lr_model.pkl', 'wb') as f:
    pickle.dump(pipeline, f)

In [18]:
X['Overall_Qual'].unique()

['Above_Average', 'Average', 'Good', 'Very_Good', 'Excellent', 'Below_Average', 'Fair', 'Poor', 'Very_Excellent', 'Very_Poor']
Categories (10, object): ['Above_Average', 'Average', 'Below_Average', 'Excellent', ..., 'Poor', 'Very_Excellent', 'Very_Good', 'Very_Poor']

In [19]:
X['Sale_Condition'].unique()

['Normal', 'Partial', 'Family', 'Abnorml', 'Alloca', 'AdjLand']
Categories (6, object): ['Abnorml', 'AdjLand', 'Alloca', 'Family', 'Normal', 'Partial']

### Moving the model to Streamlit

Now we will create a basic application using the Streamlit library [docs](https://docs.streamlit.io/). To do so, we will first create a virtual environment for the project.  Over to VSCode.

#### Final Check-in

Please update me on your final progress using this form [here](https://docs.google.com/forms/d/e/1FAIpQLSePBQjBvET71WJznwhzqYXJw9XtuqN2z6cw-PszN5sgENTD1w/viewform?usp=sharing). 